In [2]:
import hail as hl
import pandas as pd

Loading BokehJS ...

In [3]:
hl.init(default_reference = 'GRCh38',
                tmp_dir = "gs://wes-bipolar-tmp-4day/")

/opt/conda/miniconda3/lib/python3.10/site-packages/hail/context.py:347: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 37731
SPARKMONITOR_LISTENER: Application Started: application_1718122941138_0004 ...Start Time: 1718129946799


Running on Apache Spark version 3.3.2
SparkUI available at http://rye-m.c.wes-bipolar.internal:42135
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.128-eead8100a1c1
LOGGING: writing to /home/hail/hail-20240611-1819-0.2.128-eead8100a1c1.log


In [4]:
# Full manifest before sample-filtering 
MANIFEST = 'gs://2024-wgspd/files/20240523_WGSPD_final-qcd-manifest.tsv'
manifest = hl.import_table(MANIFEST, delimiter='\t',
                          key = "s", impute = True)
manifest.describe()
manifest.count()

2024-06-11 18:19:43.488 Hail: INFO: Reading table to impute column types 1) / 1]
2024-06-11 18:19:45.301 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'sex_new' as type str (imputed)
  Loading field 'sex_old' as type str (imputed)
  Loading field 'SEX' as type str (imputed)
  Loading field 'primary_disease_new' as type str (imputed)
  Loading field 'primary_disease_new_fixed' as type str (imputed)
  Loading field 'primary_disease_old' as type str (imputed)
  Loading field 'primary_disease_old_fixed' as type str (imputed)
  Loading field 'PRIMARY_DISEASE' as type str (imputed)
  Loading field 'CASECON' as type str (imputed)
  Loading field 'population_inference.pop' as type str (imputed)


----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'sex_new': str 
    'sex_old': str 
    'SEX': str 
    'primary_disease_new': str 
    'primary_disease_new_fixed': str 
    'primary_disease_old': str 
    'primary_disease_old_fixed': str 
    'PRIMARY_DISEASE': str 
    'CASECON': str 
    'population_inference.pop': str 
----------------------------------------
Key: ['s']
----------------------------------------


28554

In [5]:
# Subset dense MT before variant-filtering (by gnomad)
MT = 'gs://gnomad-subsets-2024/gnomad-v3/202403/20240328_subset_dense-callstats.mt'
mt = hl.read_matrix_table(MT)
# Filter to passing samples
mt = mt.filter_cols(hl.is_defined(manifest[mt.s]))
mt.count()

(588713326, 28554)

In [6]:
# Annotate with phenotype info
mt = mt.annotate_cols(pheno = manifest[mt.s].CASECON)
#mt = mt.annotate_cols(primary_disease = manifest[mt.s].PRIMARY_DISEASE)

### Sites of interest

In [7]:
#f = mt.filter_rows(hl.is_defined(variant_ht[mt.locus, mt.alleles]))
f = hl.filter_intervals(mt, [hl.parse_locus_interval('chr12:120291700-120292000')])
f = f.checkpoint("gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_SNV_gene-range_passing-samples.mt", overwrite = True)

2024-06-11 18:21:14.550 Hail: INFO: wrote matrix table with 366 rows and 28554 columns in 1 partition to gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_SNV_gene-range_passing-samples.mt


In [14]:
f = hl.read_matrix_table("gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_SNV_gene-range_passing-samples.mt")
f.describe()
f.count()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'pheno': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'a_index': int32
    'was_split': bool
    'AC_raw': int32
    'AN_raw': int32
    'AF_raw': float32
    'AC': int32
    'AN': int32
    'AF': float32
----------------------------------------
Entry fields:
    'GT': call
    'DP': int32
    'GQ': int32
    'MIN_DP': int32
    'RGQ': int32
    'END': int32
    'PID': str
    'SB': array<int32>
    'gvcf_info': struct {
        ClippingRankSum: float64, 
        BaseQRankSum: float64, 
        MQ: float64, 
        MQRankSum: float64, 
        MQ_DP: int32, 
        QUALapprox: int32, 
        RAW_MQ: float64, 
        ReadPosRankSum: float64, 
        VarDP: int32
    }
    'PGT': call
    'AD': array<int32>
    'PL': array<int32>
    'adj': bool
------------------

(366, 28554)

In [15]:
f = f.filter_cols(f.pheno != "OTHER")
f = f.filter_rows(f.AC < 50)
f = f.annotate_entries(non_ref = f.GT.is_non_ref())
f.count()

(336, 28554)

In [16]:
manifest.aggregate(hl.agg.counter(manifest.CASECON))

{'CASE': 7700, 'CTRL': 20854}

In [17]:
f_case_con = f.annotate_rows(case_non_ref = hl.agg.count_where(f.non_ref & (f.pheno == "CASE")),
                             con_non_ref = hl.agg.count_where(f.non_ref & (f.pheno == "CTRL")),
                             #case_ref = hl.agg.count_where(~f.non_ref & (f.pheno == "CASE")),
                             #con_ref = hl.agg.count_where(~f.non_ref & (f.pheno == "CTRL")),
                             )
f_case_con = f_case_con.annotate_rows(case_ref = 7700 - f_case_con.case_non_ref,
                                      con_ref = 20854 - f_case_con.con_non_ref
                             )
f_case_con = f_case_con.annotate_rows(fisher = hl.expr.functions.fisher_exact_test(
    hl.int(f_case_con.case_non_ref),
    hl.int(f_case_con.case_ref),
    hl.int(f_case_con.con_non_ref),
    hl.int(f_case_con.con_ref)))
h = f_case_con.rows()
h = h.order_by(h.fisher.p_value)
h.show(n = 336)

2024-06-11 18:29:51.794 Hail: INFO: Ordering unsorted dataset with network shuffle


+-----------------+-------------------------------------+------+---------+
| locus           | alleles                             | rsid | a_index |
+-----------------+-------------------------------------+------+---------+
| locus<GRCh38>   | array<str>                          | str  |   int32 |
+-----------------+-------------------------------------+------+---------+
| chr12:120291824 | ["T","G"]                           | NA   |       2 |
| chr12:120291865 | ["T","A"]                           | NA   |       1 |
| chr12:120291970 | ["C","G"]                           | NA   |       2 |
| chr12:120291875 | ["T","C"]                           | NA   |       1 |
| chr12:120291817 | ["G","A"]                           | NA   |       1 |
| chr12:120291706 | ["C","A"]                           | NA   |       1 |
| chr12:120291870 | ["C","T"]                           | NA   |       1 |
| chr12:120291944 | ["A","C"]                           | NA   |       1 |
| chr12:120291976 | ["G","C"]                           | NA   |       1 |
| chr12:120291748 | ["CTT","C"]                         | NA   |       1 |
| chr12:120291773 | ["G","A"]                           | NA   |       1 |
| chr12:120291789 | ["C","T"]                           | NA   |       2 |
| chr12:120291918 | ["C","T"]                           | NA   |       3 |
| chr12:120291740 | ["T","A"]                           | NA   |       1 |
| chr12:120291748 | ["C","T"]                           | NA   |       2 |
| chr12:120291866 | ["A","G"]                           | NA   |       1 |
| chr12:120291878 | ["C","T"]                           | NA   |       2 |
| chr12:120291956 | ["T","G"]                           | NA   |       2 |
| chr12:120291818 | ["G","A"]                           | NA   |       1 |
| chr12:120291876 | ["G","A"]                           | NA   |       1 |
| chr12:120291844 | ["T","C"]                           | NA   |       1 |
| chr12:120291777 | ["G","A"]                           | NA   |       1 |
| chr12:120291746 | ["G","A"]                           | NA   |       1 |
| chr12:120291994 | ["C","T"]                           | NA   |       1 |
| chr12:120291716 | ["T","C"]                           | NA   |       2 |
| chr12:120291926 | ["G","A"]                           | NA   |       1 |
| chr12:120291723 | ["A","C"]                           | NA   |       4 |
| chr12:120291729 | ["T","G"]                           | NA   |       2 |
| chr12:120291730 | ["A","C"]                           | NA   |       2 |
| chr12:120291733 | ["A","C"]                           | NA   |       2 |
| chr12:120291733 | ["A","G"]                           | NA   |       3 |
| chr12:120291743 | ["T","G"]                           | NA   |       1 |
| chr12:120291750 | ["T","C"]                           | NA   |       1 |
| chr12:120291751 | ["TTAAA","T"]                       | NA   |       2 |
| chr12:120291757 | ["A","C"]                           | NA   |       1 |
| chr12:120291758 | ["T","A"]                           | NA   |       1 |
| chr12:120291763 | ["T","G"]                           | NA   |       2 |
| chr12:120291779 | ["C","T"]                           | NA   |       2 |
| chr12:120291788 | ["C","T"]                           | NA   |       1 |
| chr12:120291792 | ["T","G"]                           | NA   |       3 |
| chr12:120291801 | ["T","C"]                           | NA   |       1 |
| chr12:120291805 | ["T","G"]                           | NA   |       3 |
| chr12:120291806 | ["C","G"]                           | NA   |       1 |
| chr12:120291809 | ["G","C"]                           | NA   |       2 |
| chr12:120291813 | ["T","G"]                           | NA   |       2 |
| chr12:120291817 | ["G","T"]                           | NA   |       2 |
| chr12:120291818 | ["G","GA"]                          | NA   |       2 |
| chr12:120291827 | ["T","C"]                           | NA   |       1 |
| chr12:120291827 | ["TG"

In [18]:
h = h.flatten()
h.export("gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_gene-range_passing-samples_fisher.tsv", delimiter = "\t")

2024-06-11 18:30:09.717 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-06-11 18:30:12.006 Hail: INFO: merging 2 files totalling 44.4K... + 1) / 1]
2024-06-11 18:30:12.300 Hail: INFO: while writing:
    gs://2024-wgspd/NDD_RNU4-2/20240611_NDD_RNU4-2_gene-range_passing-samples_fisher.tsv
  merge time: 293.582ms


In [19]:
variant_df = pd.DataFrame({"locus" : ["chr12:120291865"], "alleles": [["T","A"]]})
variant_ht = hl.Table.from_pandas(variant_df)
variant_ht = variant_ht.annotate(locus = hl.parse_locus(variant_ht.locus))
variant_ht = variant_ht.key_by("locus", "alleles")
#variant_ht = variant_ht.key_by("locus")
variant_ht.show()


i = hl.filter_intervals(mt, [hl.parse_locus_interval('chr12:120291865-120291866')]).persist()
i = i.filter_rows(hl.is_defined(variant_ht[i.locus, i.alleles]))
i = i.filter_cols(i.pheno != "OTHER")
i = i.annotate_entries(non_ref = i.GT.is_non_ref())
i = i.annotate_cols(carrier = hl.agg.count_where(i.non_ref))
i.aggregate_cols(hl.agg.counter(i.carrier))

,
locus,alleles
locus<GRCh38>,array<str>
chr12:120291865,"[""T"",""A""]"


2024-06-11 18:32:40.388 Hail: INFO: wrote matrix table with 2 rows and 28554 columns in 1 partition to gs://wes-bipolar-tmp-4day/persist_MatrixTableZDM10J25Zs
2024-06-11 18:32:42.255 Hail: WARN: aggregate_cols(): Aggregates over cols ordered by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2024-06-11 18:32:43.818 Hail: INFO: Coerced sorted dataset


{0: 28549, 1: 5}

In [20]:
ic = i.cols()
ic = ic.filter(ic.carrier == 1)
manifest[ic.s].show()

2024-06-11 18:32:47.013 Hail: INFO: Coerced sorted dataset
2024-06-11 18:32:48.668 Hail: INFO: Coerced sorted dataset
2024-06-11 18:32:49.297 Hail: INFO: Coerced sorted dataset


+-------------+----------------+----------------+------------+
| s           | <expr>.sex_new | <expr>.sex_old | <expr>.SEX |
+-------------+----------------+----------------+------------+
| str         | str            | str            | str        |
+-------------+----------------+----------------+------------+
| "10C114599" | "Female"       | "Female"       | "Female"   |
| "11C122337" | "Male"         | "Male"         | "Male"     |
| "MH0188521" | "Female"       | "Female"       | "Female"   |
| "MH0192991" | "Female"       | "Female"       | "Female"   |
| "MH0196843" | "Female"       | "Female"       | "Female"   |
+-------------+----------------+----------------+------------+

+---------------------------------------------+
| <expr>.primary_disease_new                  |
+---------------------------------------------+
| str                                         |
+---------------------------------------------+
| "Schizophrenia, Paranoid"                   |
| "Bipolar I with psychosis"                  |
| "Schizoaffective disorder, depressive type" |
| "Schizophrenia, Undifferentiated"           |
| "Bipolar I with psychosis"                  |
+---------------------------------------------+

+----------------------------------+
| <expr>.primary_disease_new_fixed |
+----------------------------------+
| str                              |
+----------------------------------+
| "SCZ"                            |
| "BD1"                            |
| "SCZ"                            |
| "SCZ"                            |
| "BD1"                            |
+----------------------------------+

+---------------------------------------------+
| <expr>.primary_disease_old                  |
+---------------------------------------------+
| str                                         |
+---------------------------------------------+
| "Schizophrenia, Paranoid"                   |
| "Bipolar I with psychosis"                  |
| "Schizoaffective disorder, depressive type" |
| "Schizophrenia, Undifferentiated"           |
| "Bipolar I with psychosis"                  |
+---------------------------------------------+

+----------------------------------+------------------------+----------------+
| <expr>.primary_disease_old_fixed | <expr>.PRIMARY_DISEASE | <expr>.CASECON |
+----------------------------------+------------------------+----------------+
| str                              | str                    | str            |
+----------------------------------+------------------------+----------------+
| "SCZ"                            | "SCZ"                  | "CASE"         |
| "BD1"                            | "BD1"                  | "CASE"         |
| "SCZ"                            | "SCZ"                  | "CASE"         |
| "SCZ"                            | "SCZ"                  | "CASE"         |
| "BD1"                            | "BD1"                  | "CASE"         |
+----------------------------------+------------------------+----------------+

+---------------------------------+
| <expr>.population_inference.pop |
+---------------------------------+
| str                             |
+---------------------------------+
| "afr"                           |
| "afr"                           |
| "afr"                           |
| "afr"                           |
| "afr"                           |
+---------------------------------+

In [21]:
ic.s.collect()

2024-06-11 18:32:52.007 Hail: INFO: Coerced sorted dataset


['10C114599', '11C122337', 'MH0188521', 'MH0192991', 'MH0196843']